In [6]:
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard

In [7]:
file = open("baddate.txt", encoding="utf-8")
text = file.read()

file2 = open("nietzsche.txt", encoding="utf-8")
text2 = file2.read()
text2 = text2[:103613]

text3 = text + text2

In [8]:
chars = sorted(list(set(text3)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(40, len(text3) - seqlen - 1, step):
    sentences.append(text3[i: i + seqlen + 1])
    

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1
        
        
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text3) - seqlen - 1)
    
    for diversity in [0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text3[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1200):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            
            
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

for i in range(len(model.layers)-1):
    layer = model.layers[i]
    layer.trainable = False

C:\Users\syosh\AppData\Local\Temp\ipykernel_16340\1534275677.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
C:\Users\syosh\AppData\Local\Temp\ipykernel_16340\1534275677.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), seqlen, len(chars)), dtype

Epoch 1/20
41/41 [==============================] - ETA: 0s - loss: 3.3804 - categorical_crossentropy: 3.3804 - accuracy: 0.1612
----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "r.
BRAD
I don’t know about hate.
ERIN
We"
r.
BRAD
I don’t know about hate.
ERIN
Wed we sin s ii ehe e tite i ohehitheinere ee thii oore thcoa as Ce f e ale *e ie e lod asea e se uor eonasere au onen e ioan t t oan s _rile whe oheoer aohe ton  oueio these na n and inacouad s tuein oe loi aot teId D
s t ire au aoaoof Ien oole . bio the ti oo aost io ie an io me they  oa sororero uot rss iitoe i
e s ooof an lin ode a toouse oa oathetal tore e we one alelseoeitine aoof kelioa ile fe io d to eesn oie i aran
 sie
ide in te ai in siile te doo ted  ethe t ye thee oo l. bentie bee wee. ioe eid tisois roaed thinlis tos aluo i ad toiin lt inte the a te at h lo le e ne de it is isi g tarhes I on ole te dote ael a alor ahe oin au te oo s iiis hed an t e the iae lee ere ola’the aore oes 